In [ ]:
%%capture
import sys
sys.path.append('../input/pytorch-image-models/pytorch-image-models-master')
!pip install ensemble-boxes
!pip install livelossplot

import livelossplot
import copy
import random
import math

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import random

import tqdm.notebook as tqdm

import PIL
import os
import collections

import torchvision
import torchvision.transforms as transforms


import albumentations as al
from albumentations.pytorch import ToTensorV2
import cv2


import torch
from torch.nn import functional as F
import torch.nn as nn
import torch.optim as optim

import timm

from sklearn.model_selection import train_test_split
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
import os
import random
seed = 42
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    numpy.random.seed(worker_seed)
    random.seed(worker_seed)

# Data 

MultiStage Training Strategy:
- 
Two Problems inside of the Data:
- Multiple No Findings(Solution: Train a 2 Part classifier, one training on classes or no classes, other on bbox, the ones with no findings are separated out from the rest of the dataset)
- Multiple Radiographer Readings: We will use NMS to threshold out the bounding boxes so limited duplicates occur).

No problems:
- If there is no finding, all radiographers all say no findings



In [ ]:
idx_2_class = {
    0:'Aortic enlargement',
    1:'Atelectasis',
    2:'Calcification',
    3:'Cardiomegaly',
    4:'Consolidation',
    5: 'ILD',
    6: 'Infiltration',
    7: 'Lung Opacity',
    8: 'Nodule/Mass', 
    9: 'Other lesion',
    10: 'Pleural effusion',
    11: 'Pleural thickening',
    12: 'Pneumothorax',
    13: 'Pulmonary fibrosis',
    14: 'No Finding'
}
class_2_idx = {}
for idx in idx_2_class:
    class_2_idx[idx_2_class[idx]] = idx

In [ ]:
# HYPER PARAMETERS DEFINED HERE
BATCH_SIZE = 8
TEST_BATCH_SIZE = 16

In [ ]:
# CONSTANTS DEFINED HERE
TRAIN_PATH = "../input/vinbigdata-original-image-dataset/vinbigdata/train/"
train_csv = '../input/vinbigdata-original-image-dataset/vinbigdata/train.csv'
test_csv = '../input/vinbigdata-original-image-dataset/vinbigdata/test.csv'
train_pd = pd.read_csv(train_csv)

In [ ]:
class TrainClassificationDataset(torch.utils.data.Dataset):
    '''
    This is the 2 way classification, where you classify whether a given image contains a finding or not 
    '''
    
    def __init__(self, dataframe, unique_ids, transforms, train_path):
        self.dataframe = dataframe
        self.transforms = transforms
        self.train_path = train_path
        
        self.unique_ids = unique_ids
    def cleanse_files(self, unique_ids):
        '''
        Cleanse all invalid files from the self.unique_ids
        '''
        valid_id = []
        all_files = os.listdir(self.train_path)
        for id in unique_ids:
            if id + '.jpg' in all_files:
                valid_id += [id]
        return valid_id
    def __len__(self):
        return len(self.unique_ids)
    def __getitem__(self, idx):
        # Extract an image and whether it contains an abnormality or not.
        image_id = self.unique_ids[idx]
        image_path = TRAIN_PATH + image_id + '.jpg'
        
        # Load in Image
        image = cv2.imread(image_path, 0)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # Check if classed or not(Only need to check the first one, as we perform binary classification)
        rows = self.dataframe.loc[self.dataframe.image_id.values == image_id]
        class_idx = 0
        for row in rows.iterrows():
            row = row[1]
            if row['class_id'] != 14:
                class_idx = 1
            break
            
        # Augment Image
        aug_image = self.transforms(image = image)['image']
        return torch.tensor(aug_image), class_idx

Augmentations Used:

In [ ]:
# Augmentations to use for object detection:
IMAGE_SIZE = 2048 # We will down convolution to get better performance

TRAIN_AUGMENTATIONS_CLASS = al.Compose([
    al.RandomResizedCrop(IMAGE_SIZE, IMAGE_SIZE, scale = (0.9, 0.9)),
    al.HorizontalFlip(p=0.5),
    al.RandomGamma(gamma_limit=(70, 130), p=0.3),
    al.MultiplicativeNoise(),
    al.ShiftScaleRotate(p=0.2, shift_limit=0.0025, scale_limit=0.01, rotate_limit=10),
    al.RandomBrightnessContrast(p=0.2, brightness_limit=(-0.1, 0.1), contrast_limit=(-0.1, 0.1)),
    al.HueSaturationValue(p=0.2, hue_shift_limit=0.1, sat_shift_limit=0.1, val_shift_limit=0.1),
    al.Cutout(p = 0.2),
    al.CoarseDropout(p=0.2),
    al.IAASharpen(p=0.2),
    al.Downscale(scale_min=0.7, scale_max=0.95),
    al.CLAHE(),
    al.Normalize(),
    ToTensorV2()
])

TEST_AUGMENTATIONS_CLASS = al.Compose([
    al.Resize(IMAGE_SIZE, IMAGE_SIZE),
    al.Normalize(),
    ToTensorV2()
])

In [ ]:
def unique(list_of_values):
    unique = []
    for i in list_of_values:
        if i not in unique:
            unique += [i]
    return unique

In [ ]:
# Generate Validation and Training Set
unique_ids = unique(train_pd.image_id)
train_ids, val_ids = train_test_split(unique_ids, train_size = 0.99, test_size = 0.01, random_state = 42)

In [ ]:
# Create Dataset
TrainClassDataset = TrainClassificationDataset(train_pd, train_ids, TRAIN_AUGMENTATIONS_CLASS, TRAIN_PATH);
ValClassDataset = TrainClassificationDataset(train_pd, val_ids, TEST_AUGMENTATIONS_CLASS, TRAIN_PATH);

In [ ]:
# Create Dataloaders
TrainClassDataloader = torch.utils.data.DataLoader(TrainClassDataset, batch_size = BATCH_SIZE, shuffle = True, worker_init_fn = seed_worker)
ValClassDataloader = torch.utils.data.DataLoader(ValClassDataset, batch_size = TEST_BATCH_SIZE, worker_init_fn = seed_worker)

# Part 1: Train an EfficientNetb0 for binary classification
Add Ons - Non Local Blocks, Squeeze Excite Blocks, pretty simple model

In [ ]:
class ConvBlock(nn.Module):
    def __init__(self, in_features, out_features, kernel_size, padding, groups):
        super().__init__()
        self.conv = nn.Conv2d(in_features, out_features, kernel_size, padding = padding, groups = groups)
        self.SILU = nn.SiLU(inplace = True)
        self.bn = nn.BatchNorm2d(out_features)
    def forward(self, x):
        return self.bn(self.SILU(self.conv(x)))

In [ ]:
class ConvSqueezeExcite(nn.Module):
    def __init__(self, in_features, inner_features):
        super().__init__()
        self.in_features = in_features
        self.inner_features = inner_features
        self.Squeeze = nn.Conv2d(self.in_features, self.inner_features, kernel_size = 1)
        self.Excited = nn.Conv2d(self.inner_features, self.in_features, kernel_size = 1)
        self.SILU = nn.SiLU(inplace = True)
    def forward(self, x):
        '''
        x: Tensor(B, C, H, W)
        '''
        squeezed = self.SILU(self.Squeeze(x))
        excited = torch.sigmoid(self.Excited(squeezed))
        return excited * x

In [ ]:
class RegularSE(nn.Module):
    '''
    Normal Squeeze and Excitation Block 
    '''
    def __init__(self, in_features, squeezed_features):
        super().__init__()
        self.in_features = in_features
        self.squeezed_features = squeezed_features
        self.Squeeze = nn.Linear(self.in_features, self.squeezed_features)
        self.act1 = nn.SiLU(inplace = True)
        self.Expand = nn.Linear(self.squeezed_features, self.in_features)
    def forward(self, x):
        '''
        x: Tensor(B, C, H, W)
        '''
        # Max Pool over last 2 dims
        max_pooled, _ = torch.max(x, dim = -1)
        max_pooled, _ = torch.max(max_pooled, dim = -1) # (B, C)
        # Squeeze and Excitation Network
        squeezed = self.act1(self.Squeeze(max_pooled))
        excited = torch.sigmoid(self.Expand(squeezed)).unsqueeze(-1).unsqueeze(-1)
        return excited * x
        

In [ ]:
class DownSampleConvBlock(nn.Module):
    def __init__(self, in_features, out_features, kernel_size, padding, stride, groups):
        super().__init__()
        self.conv = nn.Conv2d(in_features, out_features, kernel_size, padding = padding, stride = stride, groups = groups)
        self.SiLU = nn.SiLU(inplace = True)
        self.bn = nn.BatchNorm2d(out_features)
    def forward(self, x):
        return self.bn(self.SiLU(self.conv(x)))

In [ ]:
class BottleNeck(nn.Module):
    '''
    Squeeze Excite Residual Block as Proposed in ResNet.
    '''
    def __init__(self, input_size, inner_size, device):
        super().__init__()
        self.device = device
        self.input_size = input_size
        self.inner_size = inner_size
        self.Squeeze = ConvBlock(self.input_size, self.inner_size, 1, 0, 1)
        self.Process = ConvBlock(self.inner_size, self.inner_size, 3, 1, 1)
        self.Expand = ConvBlock(self.inner_size, self.input_size, 1, 0, 1)
        self.SE = RegularSE(self.input_size, self.input_size // 16)
        self.gamma = nn.Parameter(torch.zeros(1, device = self.device))
    def forward(self, x):
        squeezed = self.Squeeze(x)
        processed = self.Process(squeezed)
        expand = self.Expand(processed)
        excited = self.SE(expand)
        return self.gamma * excited + x

In [ ]:
class DownConvolutionBlock(nn.Module):
    '''
    Uses very cheap operations to process and downconvolve the massive image
    '''
    def __init__(self, device):
        super().__init__()
        self.device = device
        self.in_features = 3
        self.avgPool = nn.AvgPool2d(kernel_size = 5, padding = 2, stride = 2)
        self.downConv = DownSampleConvBlock(3, 5, 3, 1, 2, 1) # (2048 -> 1024)
        
        self.downConv2 = DownSampleConvBlock(8, 16, 3, 1, 2, 1) # (1024 -> 512)
        
        self.downConv3 = DownSampleConvBlock(24, 64, 3, 1, 2, 1) # (512 -> 256)
        self.process3 = nn.Sequential(*[
            BottleNeck(64, 16, self.device) for i in range(3) # A little bit of processing
        ])
        
        self.proj = nn.Sequential(*[
            ConvBlock(64, 32, 1, 0, 1),
            ConvBlock(32, 3, 1, 0, 1)])
        self.gamma = nn.Parameter(torch.zeros((1), device = self.device))
        
    def forward(self, x):
        '''
        Initial DownConvolution
        x: Tensor(B, 3, 2048, 2048)
        '''
        B, _, _, _ = x.shape
        interpolated = F.interpolate(x, (256, 256), mode = 'bilinear')
        avgPool = self.avgPool(x) # (B, 3, 1024, 1024)
        downConv = self.downConv(x) # (B, 5, 1024, 1024)
        # Concatenate Features
        concatted = torch.cat([downConv, avgPool], dim = 1) # (B, 8, 1024, 1024)
        # DownConv again 
        avgPool2 = self.avgPool(concatted) # (B, 8, 512, 512)
        downConv2 = self.downConv2(concatted) # (B, 32, 512, 512)
        concatted2 = torch.cat([downConv2, avgPool2], dim = 1) # (B, 40, 512, 512)
        # Conv Stride a Few times
        conv3 = self.process3(self.downConv3(concatted2)) # (B, 64, 256, 256)  
        proj = self.proj(conv3) # (B, 64, 128, 128)
        return proj * self.gamma + interpolated

In [ ]:
class InvertedResidualBlock(nn.Module):
    def __init__(self, in_features, inner_features, device):
        super().__init__()
        self.in_features = in_features
        self.inner_features = inner_features
        self.device = device
        self.expand = ConvBlock(self.in_features, self.inner_features, 1, 0, 1)
        self.depthwise = ConvBlock(self.inner_features, self.inner_features, 3, 1, self.inner_features)
        self.SE = RegularSE(self.inner_features, self.inner_features // 16)
        self.squeeze = ConvBlock(self.inner_features, self.in_features, 1, 0, 1)
        self.gamma = nn.Parameter(torch.zeros((1), device = self.device))
    def forward(self, x):
        expanded = self.expand(x)
        depthwise = self.depthwise(expanded)
        se = self.SE(depthwise)
        squeezed = self.squeeze(se)
        return self.gamma * squeezed + x

Model Blocks

In [ ]:
class ModifiedEfficientNetStudent(nn.Module):
    '''
    Student uses Down Convolutional Block to quickly downsample super large images.
    '''
    def freeze(self, layer):
        for parameter in layer.parameters():
            parameter.requires_grad = False
    def unfreeze(self, layer):
        for parameter in layer.parameters():
            parameter.requires_grad = True
    def __init__(self, num_classes, device, model_name = 'efficientnet_b8', drop_prob = 0.0):
        super().__init__()
        self.num_classes = num_classes 
        self.device = device
        self.model_name = model_name
        self.drop_prob = drop_prob
        self.model = timm.create_model(self.model_name, pretrained = True)
        
        # Extract Layers
        self.downsampled = DownConvolutionBlock(self.device)
        self.conv1 = self.model.conv_stem
        self.bn1 = self.model.bn1
        self.act1 = self.model.act1
        
        self.block0 = self.model.blocks[0]
        self.block1 = self.model.blocks[1]
        self.block2 = self.model.blocks[2]
        self.block3 = self.model.blocks[3]
        self.block4 = self.model.blocks[4]
        self.block5 = self.model.blocks[5]
        self.block6 = self.model.blocks[6]
        
        # Custom Layers
        self.Attention1 = RegularSE(56, 16)
        self.Attention2 = RegularSE(88, 32)
        self.Attention3 = RegularSE(248, 64)
        self.Attention4 = RegularSE(704, 256)
        
        self.layer4 = nn.Sequential(*[
            DownSampleConvBlock(704, 704, 5, 2, 2, 704), # (B, 320, 4, 4)
            ConvBlock(704, 768, 1, 0, 1)] + # (B, 768, 4, 4)
        [
            InvertedResidualBlock(768, 1536, self.device) for i in range(5)
         
        ]
        )
        self.conv2 = ConvBlock(768, 1536, 1, 0, 1)
        self.global_avg = nn.AdaptiveAvgPool2d((1, 1))
        self.dropout = nn.Dropout(self.drop_prob)
        self.Linear = nn.Linear(1536, self.num_classes)
    def forward(self, x):
        '''
        x: Tensor(B, 3, 320, 320)
        '''
        downsampled = self.downsampled(x) # (B, 3, 256, 256)
        conv1 = self.bn1(self.act1(self.conv1(downsampled))) # (B, 32, 256, 256)
        # Extract Features
        block0 = self.block0(conv1)
        block1 = self.block1(block0) # (B, 24, 64, 64) 
        attention1 = self.Attention1(block1)
        
        block2 = self.block2(attention1) # (B, 40, 32, 32)
        attention2 = self.Attention2(block2)
        
        block3 = self.block3(attention2)
        block4 = self.block4(block3) # (B, 112, 16, 16)
        attention3= self.Attention3(block4)
        
        block5 = self.block5(attention3)
        block6 = self.block6(block5) # (B, 320, 8, 8)
        attention4 = self.Attention4(block6)
        # Custom Layer 4
        layer4 = self.layer4(attention4) # (B, 512, 4, 4)
        # Classification Head.
        conv2 = self.conv2(layer4) # (B, 1536, 4, 4)
        avg_pool = torch.squeeze(self.global_avg(conv2))
        dropped_pool = self.dropout(avg_pool)
        return torch.squeeze(self.Linear(dropped_pool))

Train Alpha.

In [ ]:
class ClassificationAlpha(nn.Module):
    def __init__(self, num_classes, device):
        super().__init__()
        self.num_classes = num_classes
        self.device = device
        
        self.model = ModifiedEfficientNetStudent(self.num_classes, self.device, drop_prob = 0.0)
        
        self.optim = optim.Adam(self.model.parameters(), lr = 3e-4, weight_decay = 1e-3)
        self.lr_decay = optim.lr_scheduler.CosineAnnealingLR(self.optim, 5, eta_min = 1e-7)
        self.lr_decay2 = optim.lr_scheduler.StepLR(self.optim, 5, 0.9)
        self.criterion = nn.BCEWithLogitsLoss()
    def forward(self, x):
        self.eval()
        with torch.no_grad():
            return self.model(x)
    def training_loop(self, train_dataloader, val_dataloader, NUM_EPOCHS, display_every = 16):
        liveloss = livelossplot.PlotLosses()
        torch.cuda.empty_cache()
        
        best_val_loss = 999
        best_val_accuracy = 0
        for EPOCH in range(NUM_EPOCHS):
            self.train()
            logs = {}
            count = 0
            total_loss = 0.0
            for images, labels in train_dataloader:
                self.optim.zero_grad()
                images = images.to(torch.float32).to(self.device)
                labels = labels.to(self.device).to(torch.float32)
                logits = self.model(images)
                loss = self.criterion(logits, labels)
                loss.backward()
                self.optim.step()
                total_loss += loss.item()
                count += 1
                del images
                del labels
                del logits
                torch.cuda.empty_cache()
                if count == display_every:
                    break
            logs['loss'] = total_loss / display_every
            print(f"EPOCH: {EPOCH}, total_loss: {logs['loss']}")
            
            self.eval()
            self.lr_decay.step()
            self.lr_decay2.step()
            with torch.no_grad():
                logs['val_loss'] = 0
                logs['accuracy'] = 0
                count = 0
                for images, labels in val_dataloader:
                    images = images.to(torch.float32).to(self.device)
                    labels = labels.to(self.device).to(torch.float32)
                    pred = self.model(images)
                    logs['val_loss'] += self.criterion(pred, labels).item()
                    sigmoided = torch.sigmoid(pred)
                    ones = sigmoided >= 0.5
                    sigmoided[:] = 0
                    sigmoided[ones] = 1
                    logs['accuracy'] += torch.sum((sigmoided == labels).int()).item() / sigmoided.shape[0]
                    count += 1
                    del images
                    del labels
                    del pred
                    del sigmoided
                    torch.cuda.empty_cache();
                logs['val_loss'] /= count
                logs['accuracy'] /= count
            
            liveloss.update(logs)
            liveloss.send()
            if logs['val_loss'] < best_val_loss:
                torch.save(self.state_dict(), "./BestLoss.pth");
                best_val_loss = logs['val_loss']
            if logs['accuracy'] > best_val_accuracy:
                torch.save(self.state_dict(), "./BestAcc.pth")
                best_val_accuracy = logs['accuracy']
            print(f"EPOCH: {EPOCH}, Train_Loss: {round(logs['loss'], 4)}, Val_Loss: {round(logs['val_loss'], 4)}, val_accuracy {round(logs['accuracy'], 4)}")

In [ ]:
%%capture
Classifier = ClassificationAlpha(1, device)
Classifier.to(device)

In [ ]:
Classifier.training_loop(TrainClassDataloader, ValClassDataloader, 30, display_every = 128)

In [ ]:
torch.save(Classifier.state_dict(), "./FinalModel.pth")